<a href="https://colab.research.google.com/github/Meguazy/HACK_4DS/blob/main/notebooks_luca/HACK4DS_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Meguazy/HACK_4DS.git

Cloning into 'HACK_4DS'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 102 (delta 42), reused 4 (delta 1), pack-reused 0
Receiving objects: 100% (102/102), 11.92 MiB | 4.77 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [2]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


In [3]:
%cd /content/HACK_4DS

/content/HACK_4DS


# Initial

In [4]:
!pip install lightgbm imbalanced-learn

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import matplotlib.pyplot as plt

In [6]:
df_test = pd.read_csv("data2/test_set.csv", sep=";", quotechar="\"", decimal=",")

In [7]:
df_train = pd.read_csv("data2/train_set.csv", sep=";", quotechar="\"", decimal=",")

# Preprocessing

In [8]:
external_score_ver03_dictionary = {
    "MISSING":None,
    "A":14,
    "B":13,
    "C":12,
    "D":11,
    "E":10,
    "F":9,
    "G":8,
    "H":7,
    "I":6,
    "L":5,
    "M":4,
    "N":3,
    "O":2,
    "P":1
}

df_train = df_train.replace({"external_score_ver03": external_score_ver03_dictionary})
df_test = df_test.replace({"external_score_ver03": external_score_ver03_dictionary})

In [9]:
# Concatenate df_train and df_test
df_combined = pd.concat([df_train, df_test], axis=0)

# List of columns to encode
columns_to_encode = ['province', 'juridical_form', 'industry_sector', 'region', 'geo_area', 'decision_date']

# Instantiate LabelEncoder
label_encoders = {}

# Encode each column iteratively
for column in columns_to_encode:
    # Instantiate LabelEncoder for the current column
    label_encoders[column] = LabelEncoder()

    # Fit label encoder on combined data
    label_encoders[column].fit(df_combined[column])

    # Transform both training and testing data
    df_train[column] = label_encoders[column].transform(df_train[column])
    df_test[column] = label_encoders[column].transform(df_test[column])

In [10]:
class_columns = ['decision_date', 'company_ID', 'external_score_ver01',
       'external_score_ver02', 'late_payment_score',
       'external_score_late_payment_integrated', 'external_score_moderate',
       'external_score_adverse', 'external_score_ver03', 'age', 'province',
       'juridical_form', 'industry_sector', 'gross_margin_ratio',
       'core_income_ratio', 'cash_asset_ratio',
       'consolidated_liabilities_ratio', 'tangible_assets_ratio', 'revenues',
       'cr_available', 'region', 'geo_area', 'last_statement_age',
       'overrun_freq_a_revoca_autoliquidanti',
       'avg_tension_a_revoca_autoliquidanti',
       'std_tension_a_revoca_autoliquidanti',
       'max_tension_a_revoca_autoliquidanti',
       'last_tension_a_revoca_autoliquidanti',
       'avg_rel_used_a_revoca_autoliquidanti',
       'std_rel_used_a_revoca_autoliquidanti',
       'max_rel_used_a_revoca_autoliquidanti',
       'last_rel_used_a_revoca_autoliquidanti', 'overrun_freq_a_scadenza',
       'avg_rel_used_a_scadenza', 'std_rel_used_a_scadenza',
       'max_rel_used_a_scadenza', 'last_rel_used_a_scadenza',
       'avg_count_enti_affidanti', 'std_count_enti_affidanti',
       'max_count_enti_affidanti', 'last_count_enti_affidanti',
       'avg_count_numero_prima_info', 'std_count_numero_prima_info',
       'max_count_numero_prima_info', 'last_count_numero_prima_info']

reg_columns = ['decision_date', 'company_ID', 'external_score_ver01',
       'external_score_ver02', 'external_score_ver03', 'age', 'province',
       'juridical_form', 'industry_sector', 'gross_margin_ratio',
       'core_income_ratio', 'cash_asset_ratio',
       'consolidated_liabilities_ratio', 'tangible_assets_ratio', 'revenues',
       'cr_available', 'region', 'geo_area', 'last_statement_age',
       'overrun_freq_a_revoca_autoliquidanti',
       'avg_tension_a_revoca_autoliquidanti',
       'std_tension_a_revoca_autoliquidanti',
       'max_tension_a_revoca_autoliquidanti',
       'last_tension_a_revoca_autoliquidanti',
       'avg_rel_used_a_revoca_autoliquidanti',
       'std_rel_used_a_revoca_autoliquidanti',
       'max_rel_used_a_revoca_autoliquidanti',
       'last_rel_used_a_revoca_autoliquidanti', 'overrun_freq_a_scadenza',
       'avg_rel_used_a_scadenza', 'std_rel_used_a_scadenza',
       'max_rel_used_a_scadenza', 'last_rel_used_a_scadenza',
       'avg_count_enti_affidanti', 'std_count_enti_affidanti',
       'max_count_enti_affidanti', 'last_count_enti_affidanti',
       'avg_count_numero_prima_info', 'std_count_numero_prima_info',
       'max_count_numero_prima_info', 'last_count_numero_prima_info']

# Classificator

In [11]:
from sklearn.metrics import f1_score
import lightgbm as lgb
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(df_train[class_columns], df_train.target, test_size=0.2, random_state=42)

train_data = lgb.Dataset(X_train_class, label=y_train_class)


# Setting up parameters for LightGBM
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05,
    'random_state': 42,
    'is_unbalance': True
}

# Training the model
num_round = 3100
bst = lgb.train(params, train_data, num_round)

# Making predictions
y_pred_class = bst.predict(X_test_class, num_iteration=bst.best_iteration)

# Converting probabilities to binary predictions
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred_class]

# Calculating F1 score
f1 = f1_score(y_test_class, y_pred_binary)
print("F1 score:", f1)

[LightGBM] [Info] Number of positive: 6893, number of negative: 25135
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.295709 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6189
[LightGBM] [Info] Number of data points in the train set: 32028, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215218 -> initscore=-1.293755
[LightGBM] [Info] Start training from score -1.293755
F1 score: 0.6666666666666666


# Regressor

In [12]:
# Splitting the data into training and testing sets
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(df_train, df_train.days_to_default, test_size=0.2, random_state=42)

In [36]:
# # Scaling the features
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# X_train_scaled = scaler.fit_transform(X_train_reg[reg_columns])
# X_test_scaled = scaler.transform(X_test_reg[reg_columns])
# df_test_scaled = scaler.transform(df_test[reg_columns])
X_train_scaled = X_train_reg[reg_columns]
X_test_scaled = X_test_reg[reg_columns]
df_test_scaled = df_test[reg_columns]

In [37]:
from sklearn.metrics import mean_absolute_error as mae
from lightgbm import LGBMRegressor

model = LGBMRegressor(
    objective='regression',
    boosting_type='gbdt',
    num_leaves=51,
    learning_rate=0.18,
    n_estimators=200,
    feature_fraction=0.88,
    bagging_fraction=0.8,
    num_iterations=6000,
    bagging_freq=5,
    verbose=-1,
)
model.fit(X_train_scaled, y_train_reg)

# Make predictions on the training and validation data.
y_train_pred_reg = model.predict(X_train_scaled)
y_pred_reg = model.predict(X_test_scaled)

# Calculate and print the Root Mean Absolute Error (MAE) for training and validation predictions.
print("Training MAE: ", mae(y_train_reg, y_train_pred_reg))
print("Validation MAE: ", mae(y_test_reg, y_pred_reg))

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training MAE:  0.2361893075122995
Validation MAE:  244.61924728545512


# My regressor

In [15]:
# from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor,
#                               AdaBoostRegressor)
# from sklearn.impute import SimpleImputer
# from lightgbm import LGBMRegressor
# from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# from sklearn.model_selection import KFold, cross_val_score
# import numpy as np

# Model = []
# MAE = []
# RMSE = []
# R_sq = []
# cv = KFold(5)

# #Creating a Function to append the cross validation scores of the algorithms
# def input_scores(name, model, x, y):
#     Model.append(name)
#     MAE.append((-1) * cross_val_score(model, x, y, cv=cv,
#                                       scoring='neg_mean_absolute_error').mean())
#     RMSE.append(np.sqrt((-1) * cross_val_score(model, x, y, cv=cv,
#                                                scoring='neg_mean_squared_error').mean()))
#     R_sq.append(cross_val_score(model, x, y, cv=cv, scoring='r2').mean())

# names = ['K Neighbors Regressor', 'Decision Tree Regressor',
#          'Random Forest Regressor', 'Gradient Boosting Regressor', 'LightGBM']


# models = [
#     KNeighborsRegressor(),
#     DecisionTreeRegressor(),
#     RandomForestRegressor(),
#     GradientBoostingRegressor(),
#     LGBMRegressor(
#         objective='regression',
#         metric='mae',
#         boosting_type='gbdt',
#         num_leaves=51,
#         learning_rate=0.18,
#         n_estimators=200,
#         feature_fraction=0.88,
#         begging_fraction=0.8,
#         bagging_freq=5,
#         verbose=-1
#     )
# ]

# # Impute missing values
# imputer = SimpleImputer(strategy='median')  # You can change the strategy as needed
# X_train_imputed = imputer.fit_transform(X_train_scaled)

In [16]:
# # Running all algorithms
# for name, model in zip(names, models):
#     input_scores(name, model, X_train_imputed, y_train)

# evaluation = pd.DataFrame({'Model': Model,
#                            'MAE' : MAE,
#                            'RMSE': RMSE,
#                            'R Squared': R_sq})
# print("FOLLOWING ARE THE TRAINING SCORES: ")
# evaluation

In [17]:
# # Initialize and train the LGBMRegressor
# lgbm_regressor = LGBMRegressor(
#     objective='regression',
#     metric='mae',
#     boosting_type='gbdt',
#     num_leaves=51,
#     learning_rate=0.18,
#     n_estimators=200,
#     feature_fraction=0.88,
#     begging_fraction=0.8,
#     bagging_freq=5,
#     verbose=-1
# )

# lgbm_regressor.fit(X_train_scaled, y_train)

# # Predict on the testing set
# y_pred = lgbm_regressor.predict(X_test_scaled)

# # Calculate Mean Absolute Error (MAE)
# mae = mean_absolute_error(y_test, y_pred)
# print("Mean Absolute Error:", mae)

# Local Validation

In [38]:
df_local_val = X_test_reg.copy()

# Making predictions
y_class_local = bst.predict(df_local_val[class_columns], num_iteration=bst.best_iteration)

# Converting probabilities to binary predictions
y_class_local_binary = [1 if pred > 0.5 else 0 for pred in y_class_local]

df_local_val['default_prediction'] = y_class_local_binary

In [40]:
df_local_val['reg_prediction'] = df_local_val.apply(lambda row: model.predict([row[reg_columns].values])[0] if row['default_prediction'] == 1 else 1498, axis=1)

In [ ]:
df_local_val

In [41]:
print("Validation Local MAE: ", mae(df_local_val.days_to_default, df_local_val.reg_prediction))

Validation Local MAE:  112.3372738003468


# Submit solution

In [42]:
df_test_val = df_test.copy()

# Making predictions
y_final = bst.predict(df_test_val[class_columns], num_iteration=bst.best_iteration)

# Converting probabilities to binary predictions
y_final_binary = [1 if pred > 0.5 else 0 for pred in y_final]

df_test_val['default_prediction'] = y_final_binary

df_test_val['reg_prediction'] = df_test_val.apply(lambda row: model.predict([row[reg_columns].values])[0] if row['default_prediction'] == 1 else 1498, axis=1)

In [48]:
submission = pd.DataFrame(df_test_val['reg_prediction'])

In [50]:
submission.to_csv("file.csv", index=False, header=False)